# Exploring RESTful APIs inside Notebooks

## Presentation at [PyData Berlin Meetup, 2018-10-17](https://www.meetup.com/PyData-Berlin/events/255361308/)

This notebook contains examples for how to explore RESTful APIs inside a [Jupyter](https://jupyter.org) notebook environment. It is somewhat inspired by [Postman](https://www.getpostman.com) and aims at providing more flexibility for customising the UI when entering input and rendering output.

## Ways of Working RESTfully…

Curl and friends

In [ ]:
! curl https://xkcd.com/552/info.0.json

In [ ]:
! curl -s https://xkcd.com/552/info.0.json | jq .

In [ ]:
! http https://xkcd.com/552/info.0.json

In [ ]:
! curl -s https://xkcd.com/552/info.0.json | jq .img | sed s/\"//g

In [ ]:
! curl -s -k $(curl -s 'https://xkcd.com/552/info.0.json' | jq .img | sed s/\"//g) --output xkcd.png
! open xkcd.png

The following few examples require an HTTP API token, see https://wit.ai/docs/http/.

In [ ]:
import os
WIT_TOKEN = os.getenv('WIT_TOKEN')

In [ ]:
! curl -s -H 'Authorization: Bearer {WIT_TOKEN}' \
'https://api.wit.ai/message?q=silly+nonsense' | jq .

In [ ]:
! curl -s -H 'Authorization: Bearer {WIT_TOKEN}' \
'https://api.wit.ai/message?'\
'q="how+is+the+traffic+around+office+in+chicago"' | jq .

In [ ]:
! curl 'https://gist.githubusercontent.com/'\
'deeplook/71e9ded257cfc2d8e5e9/raw/f0cfbab5f266fcb8056e8aea046f1f222346b76b/2013.geojson'

Requests

In [ ]:
import requests
requests.get('http://python.org').headers

Postman

<img src="images/postman.png" alt="Postman UI" style="width: 90%;"/>

## Enter ipyrest

In [ ]:
from ipyrest import Api
Api('http://www.apple.com')

## Input Arguments & Parameters, etc.

This requires API access tokens as explained on https://developer.here.com/documentation.

In [ ]:
import os
from ipyrest import Api
url = 'https://1.{maptype}.maps.api.here.com/' \
      'maptile/2.1/{tiletype}/newest/{scheme}/{zoom}/{xtile}/{ytile}/{size}/{format}'
args = dict(
    maptype='traffic',
    tiletype='traffictile',
    scheme='normal.day',
    zoom='11',
    xtile='525',
    ytile='761',
    size='256',
    format='png8',
)
params = dict(
    app_id=os.getenv('HEREMAPS_APP_ID'), 
    app_code=os.getenv('HEREMAPS_APP_CODE'),
    ppi='320',
)
Api(url, args=args, params=params)

## GeoJSON Output

In [ ]:
! curl 'https://gist.githubusercontent.com/deeplook/'\
'71e9ded257cfc2d8e5e9/raw/f0cfbab5f266fcb8056e8aea046f1f222346b76b/2013.geojson'

In [ ]:
from ipyrest import Api

url = 'https://gist.githubusercontent.com/' \
      'deeplook/71e9ded257cfc2d8e5e9/raw/f0cfbab5f266fcb8056e8aea046f1f222346b76b/2013.geojson'

def post(resp):
    "Post-process response content-type since gists seem to use text/plain."
    resp.headers['Content-Type'] = 'application/vnd.geo+json'

Api(url, post_process_resp=post)

## Simple custom rendering view

In [ ]:
from ipywidgets import Textarea, Layout
from ipyrest import Api
from ipyrest.responseviews import ResponseView

class HelloWorldView(ResponseView):
    name = 'HelloWorld'
    mimetype_pats = ['text/html']
    def render(self, resp):
        layout = Layout(width='100%', height='100px')
        return Textarea(value='Hello World!', layout=layout)

url = 'https://python.org'
Api(url, additional_views=[HelloWorldView])

## Advanced rendering view

This example requires API access tokens as explained on https://developer.here.com/documentation.

In [ ]:
import os
from ipyleaflet import Map, Marker, Polyline
from ipyrest import Api
from ipyrest.responseviews import ResponseView, zoom_for_bbox

class HereIsolinesView(ResponseView):
    """
    A view for the isolines from the HERE Routing API, see
    https://developer.here.com/documentation/routing/topics/request-isoline.html.
    """
    name = 'HereIsolines'
    mimetype_pats = ['application/json']
    def render(self, resp):
        obj = resp.json()
        center = obj['response']['center']
        lat, lon = center['latitude'], center['longitude']
        m = Map(center=(lat, lon))
        m += Marker(location=(lat, lon))
        mins, maxs = [], []
        for isoline in obj['response']['isoline']:
            shape = isoline['component'][0]['shape']
            path = [tuple(map(float, pos.split(','))) for pos in shape]
            m += Polyline(locations=path, color='red', weight=2, fill=True)
            mins.append(min(path))
            maxs.append(max(path))
        m.zoom = zoom_for_bbox(*min(mins), *max(maxs))
        self.data = m
        return m
    
url = 'https://isoline.route.api.here.com' \
      '/routing/7.2/calculateisoline.json'
lat, lon = 52.5, 13.4
params = dict(
    app_id=os.getenv('HEREMAPS_APP_ID'), 
    app_code=os.getenv('HEREMAPS_APP_CODE'),
    start=f'geo!{lat},{lon}',
    mode='fastest;car;traffic:disabled',
    rangetype='time', # time/distance
    range='300,600',  # seconds/meters
    resolution='20',  # meters
    #departure='now', # 2018-07-04T17:00:00+02
)
Api(url, params=params, additional_views=[HereIsolinesView])

## 3D Output (Experimental)

This might have issues on JupyterLab, but a [classic notebook](http://localhost:8888/notebooks/pysdk/docs/postman/postmanbox-meetup.ipynb) is fine.

In [ ]:
from ipyrest import Api

def post(resp):
    "Post-proess response content-type since gists seem to have text/plain."
    resp.headers['Content-Type'] = 'application/vnd.3d+txt'
    
url = 'https://gist.githubusercontent.com/deeplook/4568232f2ca9388942aab9830ceeb21f'\
      '/raw/782da3be33080ff7c7d2bd25b7d96b6bb455d570/sample_xyz_1000.txt'
Api(url, post_process_resp=post)

As an aside, see a crude examples of ipyvolume and pptk using Lidar data in another notebook, [x_point_clouds.ipynb](x_point_clouds.ipynb).

## XKCD Variants

In [ ]:
from ipyrest import Api

url = 'https://xkcd.com/552/info.0.json'
Api(url)

In [ ]:
import requests
from ipyrest import Api

url = requests.get('https://xkcd.com/552/info.0.json').json()['img']
Api(url)

In [ ]:
import requests
from ipywidgets import Image
from ipyrest import Api
from ipyrest.responseviews import ResponseView, builtin_view_classes

class XKCDView(ResponseView):
    "Api rendering view for XKCD comics taken from XKCD JSON API."
    name = 'XKCD'
    mimetype_pats = ['application/json']
    def render(self, resp):
        return Image(value=requests.get(resp.json()['img']).content)

url = 'https://xkcd.com/552/info.0.json'
Api(url, views=builtin_view_classes + [XKCDView])

### What's the Latest Comic, BTW?

In [ ]:
url = 'https://xkcd.com/info.0.json'
Api(url, views=builtin_view_classes + [XKCDView], click_send=True)

Compare with [xkcd.com](https://xkcd.com)...

## Dynamic Views

These examples require an API access token as explained on https://docs.gitlab.com/ce/user/profile/personal_access_tokens.html.

In [ ]:
import os
TOKEN = os.environ['GITLAB_TOKEN']
server = 'https://gitlab.com/api/v4'

In [ ]:
from ipyrest import Api, ResponseView

Api(f'{server}/snippets', headers={'PRIVATE-TOKEN': TOKEN},
    cassette_path='snippets.yaml',
    timeout=20)

In [ ]:
from ipywidgets import HBox, VBox, Text, Button, Layout
from ipyrest import Api
from ipyrest.responseviews import ResponseView

class DynamicSnippetView(ResponseView):
    "ResponseView showing snippet IDs with some decent 'UI'."
    name = 'DynamicSnippetView'
    mimetype_pats = ['application/json']
    def render(self, resp):
        return VBox([
                    HBox([Text(str(snippet['id'])), 
                          Text(snippet['title']), 
                          Button(description='Delete (dummy)')]
                        ) 
            for snippet in resp.json()])

Api(f'{server}/snippets', headers={'PRIVATE-TOKEN': TOKEN},
    cassette_path='snippets.yaml',
    additional_views=[DynamicSnippetView], timeout=20)

## Skipped for Brevity

- Protobuf example
- Caching
- Timeouts
- HTTP methods other than GET
- Accessing data between response views
- Ipywidgets UI **executable w/o browser**
- Ipywidgets UI **testable via pytest**
- etc.

## Test-Suite

In [ ]:
# Run local server for testing the API in a different process locally.
import multiprocessing
import subprocess
import requests

def start_test_server():
    subprocess.check_call(['python', 'tests/api_server.py'])

url = 'http://localhost:5000/'
if requests.get(url).status_code >= 400:
    print('Starting test server...')
    multiprocessing.Process(target=start_test_server).start()
else:
    print('Test server is already running.')

In [ ]:
! pytest -s -v ../tests

## Ipyrest Under the Hood

### Architecture & Documentation

**"It's in a flux!"**

### Dependencies

- **requests**
- **ipywidgets**
- **timeout_decorator**
- **typing**
- ipyleaflet
- ipyvolume
- pandas
- pytest
- ...

### To-do-to-do-to-do

- make *true* widget
- version
- package
- docker
- binder
- swagger?

## Wrap-Up

- http://github.com/deeplook/ipyrest
- Ipyrest is an _emerging_ tool for _exploring_ APIs inside _notebooks_.
- Best used for _testing_ new APIs _quickly_ and _interactively_.
- It _might_ be also useful for providing *executable API examples* online.
- It is **#WIP #Alpha #Prototype**!
- But already useful!
- Try it!
- Contribute!
  - provide ResponseViews for your use case
  - provide examples using other APIs
  - push for missing ipywidgets issues like [this on styling](https://github.com/jupyter-widgets/ipywidgets/issues/2206)

## Q & A & T

- Questions?
- Answers!
- Thank You!